In [ ]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [ ]:
# Import cities.csv file into a dataframe
cities_df = pd.read_csv("cities.csv")

# Get rid of 'City_ID' column
cities_df = cities_df[["City", "Cloudiness", "Country", 
                       "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
cities_df.head()

In [ ]:
# Create heatmap based on humidity

# Store latitude and longitude into the variable 'coordinates'
coordinates = cities_df[["Lat", "Lng"]]

# Convert humidity to float; all data read from a csv is read as a string
humidity = cities_df["Humidity"].astype(float)

In [ ]:
# Plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
# Create a dataframe of cities that meet specific criteria to me
specs_df = cities_df.loc[(cities_df["Max Temp"] > 80) &
                         (cities_df["Humidity"] < 60) & 
                         (cities_df["Wind Speed"] < 5) | 
                         (cities_df["Wind Speed"] > 30), :].reset_index()

# Need to remove newly created index column that appears with reset_index()
specs_df = specs_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
# Preview dataframe
specs_df

In [ ]:
hotel_df = specs_df.dropna()
hotel_df

#for reference
#to see if there are nan values, code:
#nan_df = specs_df[specs_df.isna().any(axis=1)]
#nan_df

In [ ]:
# Set up additional columns to hold information
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# Want to find a hotel within 5000 meters of each (lat, lng) in my chosen destinations

# Create params dict that will be updated with a hotel name each iteration
params = {"keyword": "Hotel",
          "radius": 5000,
          "key": g_key}

# Use the lat/lng in our hotel_df to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_request = requests.get(base_url, params=params)
    
    #print the hotel_request url, avoid doing for public github repos in order to avoid exposing key
    #print(hotel_request.url)

    # convert to json
    hotel_request = hotel_request.json()
    #print(json.dumps(hotel_request, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_request["results"][0]["name"]
   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
# Preview to make sure the dataframe updates
hotel_df       

In [ ]:
# Want to drop rows with missing 'Hotel Name' data

# First fill empty 'Hotel Name' cells with NaN
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
# Preview
hotel_df

In [ ]:
# Find the rows with missing 'Hotel Name' data, for my own reference
hotel_nan = hotel_df[hotel_df.isna().any(axis = 1)]
hotel_nan

In [ ]:
# Drop the NaN rows and remove newly created 'index column' that's created with reset_index()
hotel_df = hotel_df.dropna().reset_index()
hotel_df = hotel_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", 
                     "Lng", "Max Temp", "Wind Speed", "Hotel Name"]]
# Preview
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display map
fig